In [ ]:
# Importing requests library
import random
import os
import openai
import inspect
import types
import re
import textwrap
import guardrails as gd
from rich import print
from dotenv import load_dotenv
from ast import literal_eval
import json

In [ ]:
load_dotenv()

# Prompt

In [ ]:
rail_str = """
<rail version="0.1">
<instructions>
You are AutonomousLLM."=
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code as a python function,
you shall place this code in the 'new_method' key of the JSON object you return.
You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return.
You might also receive a request to perform a task with your existing abilities. To do so,
you should look at the methods currently on yourself and make calls to your own methods to fulfil the task. 
You shall place this code in the 'execute_method' key of the JSON object you return. 
The user may ask you to generate a list of ideas for abilities to add to yourself, make them simple enough to be implemented with a single python funciton. 
You shall place this list of ideas in the 'ability_ideas' key of the JSON object you return. In this case, you can leave other keys of the JSON object is None.
Only return a JSON object and nothing else."
</instructions>

<output>
    <object name="test">
        <pythoncode
            name="new_method_init"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
        <pythoncode
            name="new_method_attach"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
        <pythoncode
            name="method_execute"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
    </object>
</output>

<prompt>
You must accomplish the task set out below:
{{task}}

Based on this task generate code and return a JSON that follows the correct schema:

{output_schema}

@json_suffix_prompt_examples
</prompt>
</rail>
"""

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)


In [ ]:
guard.base_prompt

In [ ]:
instructions = (f"You are AutonomousLLM. \n"
f"Your goal is to continously self improve. The user will provide you with abilities to add to yourself. \n"
f"When you receive such request, you must implement the requested ability in python code as a python function," 
f"you shall place this code in the 'new_method' key of the JSON object you return. \n"
f"You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return. \n"
f"You might also receive a request to perform a task with your existing abilities. To do so,"
f"you should look at the methods currently on yourself and make calls to your own methods to fulfil the task. \n" 
f"You shall place this code in the 'execute_method' key of the JSON object you return. \n"
f"The user may ask you to generate a list of ideas for abilities to add to yourself, make them simple enough to be implemented with a single python funciton.\n" 
f"You shall place this list of ideas in the 'ability_ideas' key of the JSON object you return. In this case, you can leave other keys of the JSON object is None.\n"
f"Only return a JSON object and nothing else.")

# Code

In [ ]:
class Utils:
    def __init__(self):
        pass

    def is_builtin(self, obj):
        """Check if an object is a built-in function or method."""
        if isinstance(obj, types.BuiltinFunctionType) or isinstance(
            obj, types.BuiltinMethodType
        ):
            return True
        return False

    def inspect_methods(self, obj):
        """Print the methods and code implementation of an object."""
        output_string = ""

        methods = inspect.getmembers(obj, inspect.ismethod)
        for name, method in methods:
            if not self.is_builtin(method):
                output_string += inspect.getsource(method) + "\n"

        dedented_code = textwrap.dedent(output_string)
        return dedented_code

    def convert_string_to_dict(self, string):
        cleaned_string = string.replace('\n', '')
        dictionary = json.loads(cleaned_string)
        return dictionary

In [ ]:
class CallGuard:
    guard: gd.Guard

    def __init__(self) -> None:
        pass

    def generate_code_given_task(self, task):
        self.guard = gd.Guard.from_rail("./rail/code_gen.xml")
        raw_llm_response, validated_response = guard(
            openai.ChatCompletion.create,
            prompt_params={"prompt": task},
            model="gpt-3.5-turbo",
            max_tokens=2048,
            temperature=0,
        )
        return validated_response

    
        

In [ ]:


class AutonomousLLM(Utils, CallGuard):
    def __init__(self):
        super().__init__()
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.ideas = []
        self.methods = []

    def make_api_call(self, task):
        print("Making API call.....")
        prompt = f"""


        You must accomplish the task set out below:
        {task}
        """

        new_message = {"role": "user", "content": prompt}

        self.messages.append(new_message)

        # Make a call to the GPT-3.5 Turbo API and return the generated code
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                *self.messages,
            ],
        )
        print("RESPONSE: ")
        print(response)
        print("======")
        debugRes = response 
        return response

    def generate_code_from_response(self, response):
        response_message = response.choices[0].message.content
        matches = re.findall(r"```(.*?)```", response_message, re.DOTALL)
        list_code = [match for match in matches]
        return list_code
    
    def get_response_content(self, response):
        return response.choices[0].message.content
    
    def process_response(self, response) -> dict:
        res_parsed = self.convert_string_to_dict(self.get_response_content(response))
        print(res_parsed)
        return res_parsed

    def execute_code(self, code):
        # Process the returned code and add it to AutonomousLLM
        exec(code, globals(), locals())

    def run(self, num_calls=500):
        calls_to_make = num_calls
        while calls_to_make > 0:
            calls_to_make -= 1
            # Make an API call to GPT-3.5 Turbo
            if len(self.ideas) < 1:
                res = self.make_api_call("I want you to add the ability to extract title from webpages")
            else:
                idea = self.ideas.pop(-1)
                res = self.make_api_call(f"I want you to add the {idea}")

            processed_res = self.process_response(res)

            if new_func := processed_res.get('new_method'):
                self.execute_code(textwrap.dedent(new_func))

            if attach_func := processed_res.get('attach_method'):
                self.execute_code(textwrap.dedent(attach_func))
                self.methods.append(new_func)

            if idea := processed_res.get("ability_ideas"):
                self.ideas += idea



    def use_new_ability(self):
        # Use the newly added ability in your code
        pass

In [ ]:
auto_llm = AutonomousLLM()

In [ ]:
auto_llm.extract_title("https://www.netflix.com")

In [ ]:
res = auto_llm.make_api_call("I want you to add the ability to retrieve stock prices")

In [ ]:
pres = auto_llm.process_response(res)
pres

In [ ]:
auto_llm.execute_code(pres['attach_method']['get_iphone_specs'])

In [ ]:
print(pres['attach_method']['get_iphone_specs'])

In [ ]:
get_iphone_specs("")

In [ ]:
auto_llm.execute_code(pres['attach_method'])

In [ ]:
auto_llm.__dict__

In [ ]:
auto_llm.extract_title("https://www.tryterra.co")

In [ ]:
extract_title

In [ ]:

res = auto_llm.make_api_call("Generate some ideas for abilities")


In [ ]:
res.choices[0].message.content

In [ ]:
res

In [ ]:
res = auto_llm.make_api_call("I want you to send an email")

In [ ]:
res_parsed = literal_eval(res.choices[0].message.content)
for name, res in res_parsed.items():
    print(name)
    print(res)

In [ ]:
for name, res in res_parsed.items():
    print(name)
    print(res)

In [ ]:
code_to_execute = auto_llm.process_response(res)
code_to_execute

In [ ]:
for code in code_to_execute:
    print(code)

In [ ]:
exec(code_to_execute[0])

In [ ]:
extract_title

In [ ]:
print(auto_llm.inspect_methods(auto_llm))

In [ ]:
inspect.getsource(auto_llm.__dict__['extract_title'])

In [ ]:
auto_llm.process_code(code_to_execute[1])

In [ ]:
rv = auto_llm.process_code(code_to_execute[2])
rv

In [ ]:
auto_llm.extract_title("https://www.google.com")

In [ ]:
extract_title

In [ ]:
res

In [ ]:
match = auto_llm.process_response(res)

In [ ]:
match.group(1)

In [ ]:
inspect.getmembers(auto_llm, not inspect.isbuiltin)

In [ ]:
def is_builtin(obj):
    """Check if an object is a built-in function or method."""
    if isinstance(obj, types.BuiltinFunctionType) or isinstance(obj, types.BuiltinMethodType):
        return True
    return False

def inspect_methods(obj):
    """Print the methods and code implementation of an object."""
    output_string = ""

    methods = inspect.getmembers(obj, inspect.ismethod)
    for name, method in methods:
        if not is_builtin(method):
            output_string += f"Method: {name}\n"
            output_string += inspect.getsource(method) + '\n\n'
    return output_string

In [ ]:
print(inspect_methods(auto_llm))

In [ ]:
AutonomousLLM

In [ ]:
import dis

class MyClass:
    def __init__(self):
        pass

obj = MyClass()

# Dynamically add a method using exec
exec("def new_method(self):\n    print('This is the dynamically added method')")

# Assign the method to the object
setattr(obj, "new_method", new_method)

# Retrieve the bytecode
bytecode = obj.new_method.__code__.co_code

# Disassemble the bytecode to get an approximation of the source code
disassembly = dis.dis(obj.new_method.__code__)

# Print the disassembled code
print(disassembly)

# Guardrails

## Python Code

In [ ]:
rail_str = """
<rail version="0.1">

<output>
    <pythoncode
        name="python_code"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<prompt>
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
{{leetcode_problem}}

@complete_json_suffix</prompt>

</rail>
"""

In [ ]:
import guardrails as gd

from rich import print

guard = gd.Guard.from_rail_string(rail_str)

In [ ]:
print(guard.base_prompt)

In [ ]:
import openai

leetcode_problem = """
Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.
"""

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={"leetcode_problem": leetcode_problem},
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

In [ ]:
raw_llm_response

In [ ]:
print(validated_response)

## ChatCompletions

In [7]:
import guardrails as gd

from rich import print

content = gd.docs_utils.read_pdf("./chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

Chase Credit Card Document:

2/25/23, 7:59 PM about:blank
about:blank 1/4
PRICING INFORMATION
INTEREST RATES AND INTEREST CHARGES
Purchase Annual
Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on the Prim
...

In [13]:
rail_str = """
<rail version="0.1">

<output>
    <pythoncode
        name="new_method"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
    <pythoncode
        name="attach_method"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<instructions>
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

@json_suffix_prompt_examples
</instructions>


<prompt>
You are AutonomousLLM
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code as a python function, you shall place this code in the 'new_method' key of the JSON object you return.
You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return.
You shall place this code in the 'execute_method' key of the JSON object you return. 
Only return a JSON object and nothing else.

The user request is:
{{prompt}}

@xml_prefix_prompt

{output_schema}
</prompt>

</rail>
"""

In [14]:
guard = gd.Guard.from_rail_string(rail_str)

In [15]:
print(guard.base_prompt)

You are AutonomousLLM
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code as a python function, you 
shall place this code in the 'new_method' key of the JSON object you return.
You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key 
of the JSON object you return.
You shall place this code in the 'execute_method' key of the JSON object you return. 
Only return a JSON object and nothing else.

The user request is:
{prompt}


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <pythoncode name="new_method" format="bug-free-python"/>
    <pythoncode name="attach_method" format="bug-free-python"/>
</output>

In [16]:
print(guard.instructions.source)

You are a helpful assistant only capable of communicating with valid JSON, and no other text.


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{{'foo': 'example one'}}`
- `<list name='bar'><string format='upper-case' /></list>` => `{{"bar": ['STRING ONE', 'STRING TWO', etc.]}}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{{'baz': {{'foo': 'Some String', 'index': 1}}}}`

In [21]:
import openai

raw_llm_response, validated_response = guard(
    openai.ChatCompletion.create,
    prompt_params={"prompt": "Add the ability to detect sentiment of webpage"},
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

I am being run! in guard
HEY I'M HERE
I am being called!
{'new_method': "def detect_sentiment(url):\n    from textblob import TextBlob\n    import requests\n    \n    response = requests.get(url)\n    blob = TextBlob(response.text)\n    \n    sentiment = blob.sentiment.polarity\n    \n    if sentiment > 0:\n        return 'Positive'\n    elif sentiment < 0:\n        return 'Negative'\n    else:\n        return 'Neutral'\n", 'attach_method': "setattr(self, 'detect_sentiment', detect_sentiment)", 'execute_method': None}


In [22]:
print(validated_response)

{
    'new_method': "def detect_sentiment(url):\n    from textblob import TextBlob\n    import requests\n    \n    
response = requests.get(url)\n    blob = TextBlob(response.text)\n    \n    sentiment = blob.sentiment.polarity\n  
\n    if sentiment > 0:\n        return 'Positive'\n    elif sentiment < 0:\n        return 'Negative'\n    else:\n
return 'Neutral'\n",
    'attach_method': "setattr(self, 'detect_sentiment', detect_sentiment)",
    'execute_method': None
}

In [9]:
raw_llm_response

NameError: name 'raw_llm_response' is not defined

In [ ]:
validated_response

In [ ]:
print(raw_llm_response)